<a href="https://colab.research.google.com/github/Shripad1020/Model-Deployment-to-GCP/blob/master/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Hardware accelerator: GPU
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd
%cd /content/drive/MyDrive/kaggle

/content
/content/drive/MyDrive/kaggle


In [4]:
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [5]:
# Setup data inputs
IMG_SIZE = (224, 224)

def create_data_loaders(train_dir, test_dir, image_size=IMG_SIZE):
  train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)

  test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  return train_data, test_data

In [6]:
# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [7]:
# Setup input shape and base model, freezing the base model layers
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
  base_model.trainable = False

  # Create input layer
  inputs = layers.Input(shape=input_shape, name="input_layer")

  # Add in data augmentation Sequential model as a layer
  x = data_augmentation(inputs)

  # Give base_model inputs (after augmentation) and don't train it
  x = base_model(x, training=False)

  # Pool output features of base model
  x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

  # Put a dense layer on as the output
  outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

  # Make a model with inputs and outputs
  model = keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  
  return model

16711680/16705208 [==============================] - 0s 0us/step


In [8]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=False):
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  if scale:
    return img/255.
  else:
    return img

In [9]:
# Unzip the downloaded file
def unzip_data(filename):
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [10]:
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [9]:
# Get data
import zipfile
unzip_data("10_food_classes_all_data.zip")

## Model 1 (10 classes)

In [11]:
# Create BatchDataset
train_data, test_data = create_data_loaders(train_dir="10_food_classes_all_data/train/",
                                            test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [12]:
train_data

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [13]:
# Create model
model_1 = create_model(num_classes=len(train_data.class_names))

# Fit the model
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)), # validate for less steps
                    # Track model training logs
                    callbacks=[create_tensorboard_callback("transfer_learning", "all_data_aug")])

Saving TensorBoard log files to: transfer_learning/all_data_aug/20210511-190905
Epoch 1/5
235/235 [==============================] - 168s 565ms/step - loss: 1.4190 - accuracy: 0.5644 - val_loss: 0.5259 - val_accuracy: 0.8520
Epoch 2/5
235/235 [==============================] - 108s 457ms/step - loss: 0.7244 - accuracy: 0.7752 - val_loss: 0.4299 - val_accuracy: 0.8635
Epoch 3/5
235/235 [==============================] - 98s 414ms/step - loss: 0.6229 - accuracy: 0.8030 - val_loss: 0.3974 - val_accuracy: 0.8684
Epoch 4/5
235/235 [==============================] - 96s 403ms/step - loss: 0.5737 - accuracy: 0.8216 - val_loss: 0.4115 - val_accuracy: 0.8734
Epoch 5/5
235/235 [==============================] - 91s 383ms/step - loss: 0.5485 - accuracy: 0.8327 - val_loss: 0.4032 - val_accuracy: 0.8701


In [14]:
# Get an image Tensor
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg

--2021-05-11 19:18:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874848 (2.7M) [image/jpeg]
Saving to: ‘03-pizza-dad.jpeg’

03-pizza-dad.jpeg   100%[===================>]   2.74M  --.-KB/s    in 0.1s    

2021-05-11 19:18:29 (26.3 MB/s) - ‘03-pizza-dad.jpeg’ saved [2874848/2874848]



In [15]:
# Classes our model is trained on
class_names = train_data.class_names
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [16]:
# Preprocess image
pizza_img = load_and_prep_image("03-pizza-dad.jpeg")
pizza_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 73.625,  76.75 ,  67.125],
        [114.   , 122.   , 101.   ],
        [146.875, 151.875, 129.875],
        ...,
        [ 14.5  ,  17.5  ,  10.5  ],
        [ 14.25 ,  19.25 ,  12.25 ],
        [ 19.75 ,  22.75 ,  15.75 ]],

       [[239.125, 243.625, 246.125],
        [225.375, 232.125, 234.875],
        [240.   , 245.   , 244.5  ],
        ...,
        [ 11.   ,  14.   ,   7.   ],
        [ 20.   ,  23.   ,  16.   ],
        [ 20.875,  25.875,  18.875]],

       [[ 32.5  ,  34.5  ,  31.5  ],
        [ 44.625,  44.5  ,  42.375],
        [ 33.   ,  38.   ,  34.   ],
        ...,
        [  8.75 ,  13.25 ,   6.25 ],
        [ 14.875,  17.875,  10.875],
        [ 13.625,  20.625,  12.625]],

       ...,

       [[ 61.875,  40.875,  19.875],
        [ 60.   ,  42.   ,  22.   ],
        [ 61.   ,  43.   ,  21.   ],
        ...,
        [134.5  ,  96.125,  60.75 ],
        [104.875,  69.375,  43.125],
        [106.25 ,  75.2

In [17]:
# Make predictions
pizza_expanded = tf.expand_dims(pizza_img, axis=0) # expand image dimensions (224, 224, 3) -> (1, 224, 224, 3)
pred = model_1.predict(pizza_expanded)
pred

array([[4.6374924e-03, 4.9962833e-05, 2.2786309e-04, 3.9483485e-04,
        5.7283623e-06, 2.3240937e-06, 9.9304044e-01, 1.5828549e-04,
        1.1444876e-04, 1.3685966e-03]], dtype=float32)

In [18]:
class_names = ['chicken_curry',
				 'chicken_wings',
				 'fried_rice',
				 'grilled_salmon',
				 'hamburger',
				 'ice_cream',
				 'pizza',
				 'ramen',
				 'steak',
				 'sushi']

In [19]:
# Check the predicted class
class_names[tf.argmax(pred[0])]

'pizza'

In [20]:
# Save model_1
model_1.save("efficientnet_model_1_10_classes")

INFO:tensorflow:Assets written to: efficientnet_model_1_10_classes/assets


In [21]:
#model_1 = tf.keras.models.load_model("efficientnet_model_1_10_classes")

In [22]:
from google.colab import auth
auth.authenticate_user()

In [23]:
# Install and initialize the gcloud command line tool - https://cloud.google.com/sdk/docs/install
# (this will give you access to gcloud tools within Colab)
!curl https://sdk.cloud.google.com | bash 1> /dev/null
!gcloud init

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0   6239      0 --:--:-- --:--:-- --:--:--  6239
######################################################################## 100.0%
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
######################################################################## 100.0%
mkdir -p /root
tar -C /root -zxvf /tmp/tmp.lIptK8zlwj/google-cloud-sdk.tar.gz
/root/google-cloud-sdk/install.sh
Beginning update. This process may take several minutes.


Your current Cloud SDK version is: 340.0.0
Installing components from version: 340.0.0

┌─────────────────────────────────────────────────────────────────────────────┐
│                     These components will be installed.                     │
├─────────────────────────────────────────────────────┬────────────┬──────────┤
│        

In [24]:
# Copy model to bucket - https://cloud.google.com/storage/docs/uploading-objects#gsutil 
# Use "-r" for folders (r stands for recursive)
!gsutil cp -r efficientnet_model_1_10_classes gs://iot_cp_image_classification

Copying file://efficientnet_model_1_10_classes/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://efficientnet_model_1_10_classes/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://efficientnet_model_1_10_classes/variables/variables.index [Content-Type=application/octet-stream]...
\ [3 files][ 22.1 MiB/ 22.1 MiB]                                                
Operation completed over 3 objects/22.1 MiB.                                     


## Model 2 (11 classes + Non-Food images)

In [11]:
# Download data
!wget https://storage.googleapis.com/ml_deploymet_lec_bucket_0/11_food_classes_plus_not_food_class_all_data.zip 
unzip_data("11_food_classes_plus_not_food_class_all_data.zip")

--2021-05-11 20:50:07--  https://storage.googleapis.com/ml_deploymet_lec_bucket_0/11_food_classes_plus_not_food_class_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.81.208, 142.250.73.208, 142.250.65.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.81.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1053780900 (1005M) [application/zip]
Saving to: ‘11_food_classes_plus_not_food_class_all_data.zip’

11_food_classes_plu 100%[===================>]   1005M  68.3MB/s    in 15s     

2021-05-11 20:50:22 (67.8 MB/s) - ‘11_food_classes_plus_not_food_class_all_data.zip’ saved [1053780900/1053780900]



In [12]:
# Create data loaders
train_data, test_data = create_data_loaders(train_dir="11_food_classes_plus_not_food_class_all_data/train/",
                                            test_dir="11_food_classes_plus_not_food_class_all_data/test/")

Found 14343 files belonging to 12 classes.
Found 4273 files belonging to 12 classes.


In [13]:
# Create model
model_2 = create_model(num_classes=len(train_data.class_names))

# Fit model
history_2 = model_2.fit(train_data,
                        epochs=5,
                        validation_data=test_data,
                        validation_steps=int(0.25*len(test_data)))

Epoch 1/5
449/449 [==============================] - 195s 352ms/step - loss: 1.2466 - accuracy: 0.6470 - val_loss: 0.5501 - val_accuracy: 0.8665
Epoch 2/5
449/449 [==============================] - 121s 268ms/step - loss: 0.6099 - accuracy: 0.8240 - val_loss: 0.4723 - val_accuracy: 0.8703
Epoch 3/5
449/449 [==============================] - 112s 248ms/step - loss: 0.5457 - accuracy: 0.8394 - val_loss: 0.4598 - val_accuracy: 0.8778
Epoch 4/5
449/449 [==============================] - 106s 234ms/step - loss: 0.4995 - accuracy: 0.8527 - val_loss: 0.4388 - val_accuracy: 0.8769
Epoch 5/5
449/449 [==============================] - 99s 220ms/step - loss: 0.4710 - accuracy: 0.8623 - val_loss: 0.4194 - val_accuracy: 0.8769


In [14]:
# Save model_2
model_2.save("efficientnet_model_2_12_classes")

INFO:tensorflow:Assets written to: efficientnet_model_2_12_classes/assets


In [15]:
model_2.evaluate(test_data)

134/134 [==============================] - 22s 158ms/step - loss: 0.4081 - accuracy: 0.8888


[0.40807825326919556, 0.8888368606567383]

In [ ]:
# Auth Colab
from google.colab import auth
auth.authenticate_user()
!curl https://sdk.cloud.google.com | bash 1> /dev/null
!gcloud init

In [17]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: shripad.kulkarni18@vit.edu
  project: iot-course-project-312117

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] shripad.kulkarni18@vit.edu
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [shripad.kulkarni18

In [18]:
!gsutil cp -r efficientnet_model_2_12_classes gs://iot_cp_image_classification

Copying file://efficientnet_model_2_12_classes/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://efficientnet_model_2_12_classes/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://efficientnet_model_2_12_classes/variables/variables.index [Content-Type=application/octet-stream]...
- [3 files][ 22.1 MiB/ 22.1 MiB]                                                
Operation completed over 3 objects/22.1 MiB.                                     
